In [7]:
import numpy as np
import pandas as pd
import random

# --- CONFIGURATION MATCHING YOUR TOPOLOGY ---
# Real Topology: Core=1-4, Pods start at 5
# Edge Switches: [7, 8, 11, 12, 15, 16, 19, 20]
NUM_SWITCHES = 20
NUM_HOSTS = 16
LINK_CAPACITY = 100.0 

class MockRyuController:
    """
    Digital Twin calibrated to 'sat2iptopo.py'.
    IDs: Core(1-4), Agg/Edge interleaved.
    Ports: Edge Uplinks=1,2 | Agg Uplinks=1,2.
    """
    def __init__(self, traffic_file="Dataset/Synthesis/synthetic_traffic_16hosts_SPARSE.csv"):
        self.traffic_data = pd.read_csv(traffic_file, header=None).values
        self.time_step = 0
        self.max_steps = self.traffic_data.shape[0]
        
        # State Arrays
        self.switch_utilization = {} 
        self.switch_latency = {}
        self._reset_stats()
        
        # Build the exact map from your Mininet script
        self.topo_links, self.edge_switches = self._build_real_topology()
        self.flow_table = {} 

    def _reset_stats(self):
        for sw in range(1, NUM_SWITCHES + 1):
            self.switch_utilization[sw] = {p: 0.0 for p in range(1, 5)}
            self.switch_latency[sw] = {p: 1.0 for p in range(1, 5)}

    def _build_real_topology(self):
        """
        Recreates logic from sat2iptopo.py
        """
        topo = {}
        edge_sws = []
        
        def add_link(u, u_port, v, v_port):
            topo[(u, u_port)] = (v, v_port)
            topo[(v, v_port)] = (u, u_port)

        # 1. DEFINE SWITCH IDS
        # Core: 1, 2, 3, 4
        core_switches = [1, 2, 3, 4]
        
        sw_id_counter = 5 # Start Pods at 5
        
        # 2. BUILD PODS (Logic matches your loop)
        for pod in range(4):
            # Aggregation Switches (2 per pod)
            agg_switches = []
            for i in range(2):
                agg_id = sw_id_counter
                agg_switches.append(agg_id)
                sw_id_counter += 1
                
                # Agg Uplinks to Core (Ports 1 & 2 based on addLink order)
                # Agg 1 connects to Core 1, 2
                # Agg 2 connects to Core 3, 4
                start_core = i * 2
                add_link(agg_id, 1, core_switches[start_core], pod + 1)
                add_link(agg_id, 2, core_switches[start_core+1], pod + 1)

            # Edge Switches (2 per pod)
            for i in range(2):
                edge_id = sw_id_counter
                edge_sws.append(edge_id)
                sw_id_counter += 1
                
                # Edge Uplinks to Agg (Ports 1 & 2)
                # Edge connects to BOTH Agg switches in the pod
                add_link(edge_id, 1, agg_switches[0], 3 if i==0 else 4) 
                add_link(edge_id, 2, agg_switches[1], 3 if i==0 else 4)
                
                # Ports 3 & 4 are Hosts (Implicit in simulation)

        return topo, edge_sws

    def get_stats(self):
        if self.time_step >= self.max_steps: self.time_step = 0
        current_traffic = self.traffic_data[self.time_step]
        self._reset_stats()
        
        t_matrix = current_traffic.reshape(NUM_HOSTS, NUM_HOSTS)
        for src in range(NUM_HOSTS):
            for dst in range(NUM_HOSTS):
                volume = t_matrix[src, dst]
                if volume > 0:
                    self._route_packet(src, dst, volume)
        
        # Calculate Latency
        for sw in range(1, NUM_SWITCHES + 1):
            for p in range(1, 5):
                util = self.switch_utilization[sw][p] / LINK_CAPACITY
                self.switch_utilization[sw][p] = util
                if util > 1.0:
                    self.switch_latency[sw][p] = 1.0 + (util - 1.0) * 500
        
        return self.switch_utilization, self.switch_latency

    def _route_packet(self, src_host, dst_host, volume):
        # Map Host ID (0-15) to Real Edge Switch ID
        # Pod 0 hosts 0,1 -> Edge 7. Hosts 2,3 -> Edge 8.
        # This formula maps 0->7, 1->7, 2->8, 3->8...
        pod = src_host // 4
        sub_idx = (src_host % 4) // 2
        src_sw = 5 + (pod * 4) + 2 + sub_idx 
        
        # Same for Dest
        pod_dst = dst_host // 4
        sub_idx_dst = (dst_host % 4) // 2
        dst_sw = 5 + (pod_dst * 4) + 2 + sub_idx_dst
        
        if src_sw == dst_sw: return # Local traffic

        # --- 1. EDGE SWITCH (Uplink Choice) ---
        # Ports 1 & 2 are Uplinks now!
        flow_key = (src_sw, src_host, dst_host)
        if flow_key in self.flow_table:
            edge_out = self.flow_table[flow_key]
        else:
            # Default Hash
            edge_out = 1 if (src_host + dst_host) % 2 == 0 else 2
        
        self.switch_utilization[src_sw][edge_out] += volume
        
        # --- 2. AGGREGATION SWITCH ---
        next_node = self.topo_links.get((src_sw, edge_out))
        if not next_node: return
        agg_sw, _ = next_node
        
        # Agg decision (Ports 1 or 2 go to Core)
        flow_key_agg = (agg_sw, src_host, dst_host)
        if flow_key_agg in self.flow_table:
            agg_out = self.flow_table[flow_key_agg]
        else:
            agg_out = 1 if (src_host + dst_host) % 2 == 0 else 2
            
        self.switch_utilization[agg_sw][agg_out] += volume
        
        # --- 3. CORE SWITCH ---
        next_node_core = self.topo_links.get((agg_sw, agg_out))
        if not next_node_core: return
        core_sw, core_in = next_node_core
        self.switch_utilization[core_sw][core_in] += volume

    def mod_flow(self, dpid, out_port):
        """
        Agent controls Edge Switch Uplinks (Ports 1 or 2).
        """
        current_traffic = self.traffic_data[self.time_step]
        t_matrix = current_traffic.reshape(NUM_HOSTS, NUM_HOSTS)
        
        biggest_vol = 0
        target_flow = None
        
        for src in range(NUM_HOSTS):
            for dst in range(NUM_HOSTS):
                vol = t_matrix[src, dst]
                if vol > 0:
                    # Map Host to Switch to see if this flow starts here
                    pod = src // 4
                    sub = (src % 4) // 2
                    src_real_sw = 5 + (pod * 4) + 2 + sub
                    
                    if src_real_sw == dpid:
                        if vol > biggest_vol:
                            biggest_vol = vol
                            target_flow = (src, dst)
        
        if target_flow:
            self.flow_table[(dpid, target_flow[0], target_flow[1])] = out_port
            return True
        return False

    def tick(self):
        self.time_step += 1

# --- VERIFICATION TEST ---
if __name__ == "__main__":
    env = MockRyuController()
    print("✅ Calibrated Environment Loaded.")
    print(f"   Edge Switches are: {env.edge_switches}")
    print(f"   (Should be [7, 8, 11, 12, 15, 16, 19, 20])")
    
    # Test Routing
    print("\n🧪 Testing Path from Host 0 (Switch 7) -> Host 15 (Switch 20)")
    # Force traffic
    env.traffic_data[0] = np.zeros(256) # Clear
    env.traffic_data[0][0*16 + 15] = 80 # Host 0 -> 15 (80Mbps)
    
    util, _ = env.get_stats()
    
    # Check Edge 7 Uplinks (Port 1 or 2)
    p1 = util[7][1]
    p2 = util[7][2]
    print(f"   Edge 7 Uplinks: Port 1={p1}, Port 2={p2}")
    
    if p1 > 0 or p2 > 0:
        print("   ✅ Traffic is correctly using Uplink Ports 1/2.")
    else:
        print("   ❌ Error: Traffic not leaving Edge Switch.")

✅ Calibrated Environment Loaded.
   Edge Switches are: [7, 8, 11, 12, 15, 16, 19, 20]
   (Should be [7, 8, 11, 12, 15, 16, 19, 20])

🧪 Testing Path from Host 0 (Switch 7) -> Host 15 (Switch 20)
   Edge 7 Uplinks: Port 1=0.0, Port 2=0.8
   ✅ Traffic is correctly using Uplink Ports 1/2.


In [8]:
import time
import numpy as np

# --- 1. INITIALIZE THE DIGITAL TWIN ---
print("🎬 STARTING CALIBRATION TEST (10 Seconds)...")
print("   Target Topology: sat2iptopo.py (Core 1-4, Edge 7+)")
print("-----------------------------------------------------")

try:
    # Load the environment
    # Ensure the filename matches your actual generated file
    mock = MockRyuController("Dataset/Synthesis/synthetic_traffic_16hosts_SPARSE.csv")
    print("✅ Environment Loaded.")
except NameError:
    print("❌ Error: Please run the MockRyuController class cell first!")
    exit()

# --- 2. RUN SIMULATION LOOP ---
history_path_a = []
history_path_b = []

for step in range(10):
    print(f"\n⏱️  [Time {step+1}] Processing Traffic...")
    
    # A. Get Stats (Physics Engine)
    util, lat = mock.get_stats()
    
    # B. Analyze "Path A" vs "Path B" (Summing Pairs)
    # Path A: Traffic via Agg 5 -> goes to Core 1 AND Core 2
    load_path_a = (util[1][1] + util[2][1]) * 100 
    
    # Path B: Traffic via Agg 6 -> goes to Core 3 AND Core 4
    load_path_b = (util[3][1] + util[4][1]) * 100
    
    history_path_a.append(load_path_a)
    history_path_b.append(load_path_b)

    # C. Print Status in English
    print(f"   🌊 Network Status:")
    # Check Edge Switch 7 (where Host 0 lives)
    print(f"      Edge Switch 7 (Host 0): {util[7][1]*100:.1f}% on Uplink 1 | {util[7][2]*100:.1f}% on Uplink 2")
    print(f"      Path A (Cores 1+2):     {load_path_a:.1f}% Load")
    print(f"      Path B (Cores 3+4):     {load_path_b:.1f}% Load")

    # --- D. THE INTERVENTION (At Second 5) ---
    if step == 4:
        print("\n   🚨 INTERVENTION! Agent is taking action...")
        print("   Goal: Move traffic from Edge Switch 7 -> Uplink 2 (Path B).")
        
        # We target Edge Switch 7 (Where Host 0 lives)
        # We force it to use Port 2 (which leads to Agg 6 -> Path B)
        success = mock.mod_flow(dpid=7, out_port=2)
        
        if success:
            print("   ✅ Command Accepted: Rerouting Elephant Flow on Switch 7 to Port 2.")
        else:
            print("   ⚠️  Warning: No major flow found on Switch 7 to reroute.")

    # E. Tick Clock
    mock.tick()

# --- 3. FINAL REPORT ---
print("\n-----------------------------------------------------")
print("📊 TEST SUMMARY:")
avg_before = np.mean(history_path_a[:5])
avg_after  = np.mean(history_path_a[5:])
avg_dest_after = np.mean(history_path_b[5:])

print(f"   Avg Load Path A (Before Action): {avg_before:.1f}%")
print(f"   Avg Load Path A (After Action):  {avg_after:.1f}%  <-- Should decrease")
print(f"   Avg Load Path B (After Action):  {avg_dest_after:.1f}%  <-- Should increase")

if avg_after < avg_before and avg_dest_after > 10.0:
    print("\n✅ SUCCESS: The logic matches 'sat2iptopo.py' perfectly.")
    print("   The Agent correctly identified Edge Switch 7 and moved traffic to the alternate Core Path.")
else:
    print("\n❌ ISSUE: Traffic did not move significantly. Check if Host 0 is generating enough traffic.")

🎬 STARTING CALIBRATION TEST (10 Seconds)...
   Target Topology: sat2iptopo.py (Core 1-4, Edge 7+)
-----------------------------------------------------
✅ Environment Loaded.

⏱️  [Time 1] Processing Traffic...
   🌊 Network Status:
      Edge Switch 7 (Host 0): 88.7% on Uplink 1 | 76.8% on Uplink 2
      Path A (Cores 1+2):     163.0% Load
      Path B (Cores 3+4):     152.8% Load

⏱️  [Time 2] Processing Traffic...
   🌊 Network Status:
      Edge Switch 7 (Host 0): 36.7% on Uplink 1 | 19.5% on Uplink 2
      Path A (Cores 1+2):     137.1% Load
      Path B (Cores 3+4):     92.2% Load

⏱️  [Time 3] Processing Traffic...
   🌊 Network Status:
      Edge Switch 7 (Host 0): 31.4% on Uplink 1 | 6.2% on Uplink 2
      Path A (Cores 1+2):     98.0% Load
      Path B (Cores 3+4):     28.8% Load

⏱️  [Time 4] Processing Traffic...
   🌊 Network Status:
      Edge Switch 7 (Host 0): 8.6% on Uplink 1 | 59.4% on Uplink 2
      Path A (Cores 1+2):     53.9% Load
      Path B (Cores 3+4):     120.2% L

In [9]:
import time
import numpy as np
import pandas as pd

# --- 1. SETUP ---
print("🎬 STARTING LIVE TRAFFIC REROUTE TEST...")
print("   (Using ACTUAL DATASET: synthetic_traffic_16hosts_SPARSE.csv)")

try:
    # Load the environment with the specific dataset
    dataset_path = "Dataset/Synthesis/synthetic_traffic_16hosts_SPARSE.csv"
    mock = MockRyuController(dataset_path)
    print("✅ Environment Loaded.")
    
    # --- AUTO-TARGETING LOGIC ---
    # Since the data is sparse, Host 0 might be silent. 
    # Let's find a Host that actually has traffic in the first few seconds.
    print("\n🔍 Scanning dataset for an active Elephant Flow...")
    
    first_row = mock.traffic_data[0].reshape(16, 16)
    max_flow = 0
    target_src, target_dst = 0, 0
    
    for r in range(16):
        for c in range(16):
            if first_row[r][c] > max_flow:
                max_flow = first_row[r][c]
                target_src, target_dst = r, c
                
    print(f"   🎯 Found Elephant Flow: Host {target_src} -> Host {target_dst} ({max_flow:.2f} Mbps)")
    
    # Calculate which Switch belongs to this Source Host
    # Logic: Pod = host // 4, Sub = (host % 4) // 2
    # Switch ID = 5 + (pod * 4) + 2 + sub
    pod = target_src // 4
    sub = (target_src % 4) // 2
    target_switch_id = 5 + (pod * 4) + 2 + sub
    
    print(f"   📍 Targeting Edge Switch: {target_switch_id}")

except NameError:
    print("❌ Error: Please run the MockRyuController class cell first!")
    exit()
except FileNotFoundError:
    print(f"❌ Error: Could not find {dataset_path}")
    exit()

# --- 2. RUN SIMULATION LOOP ---
# We track the Specific Switch where the Elephant lives
print(f"\n   [Time]  |  Switch {target_switch_id} Port 1 (Path A)  |  Switch {target_switch_id} Port 2 (Path B)  |  Controller Action")
print("   " + "-"*85)

# We run for 10 steps. Note: Real traffic fluctuates, so values will change naturally!
for step in range(10):
    
    # A. Physics Engine (Get Stats)
    util, lat = mock.get_stats()
    
    # B. Monitor the Target Switch Uplinks
    # Port 1 -> Goes to Agg 1 -> Core 1/2 (Path A)
    # Port 2 -> Goes to Agg 2 -> Core 3/4 (Path B)
    load_p1 = util[target_switch_id][1] * 100 
    load_p2 = util[target_switch_id][2] * 100

    # C. Prepare Action Message
    action_msg = ""
    
    # --- D. THE INTERVENTION (At Second 5) ---
    if step == 5:
        action_msg = "🚨 REROUTE COMMAND SENT!"
        # Force the switch to move the Elephant to Port 2
        mock.mod_flow(dpid=target_switch_id, out_port=2)

    # Print Table Row
    # We use Fixed Width formatting so the table looks like a Controller Dashboard
    print(f"   [{step+1:02d}s]   |         {load_p1:6.1f}%          |         {load_p2:6.1f}%          |  {action_msg}")

    # E. Tick Clock
    mock.tick()

# --- 3. VERDICT ---
print("\n-----------------------------------------------------")
# We check if traffic appeared on Port 2 after step 5
if load_p2 > 0:
    print("✅ SUCCESS: Traffic moved to Port 2 (Path B).")
    print(f"   The Agent successfully manipulated Edge Switch {target_switch_id}.")
else:
    print("⚠️  WARNING: Traffic did not appear on Port 2.")
    print("   Possible reasons:")
    print("   1. The flow naturally ended in the dataset (it was a short burst).")
    print("   2. The reroute logic failed.")

🎬 STARTING LIVE TRAFFIC REROUTE TEST...
   (Using ACTUAL DATASET: synthetic_traffic_16hosts_SPARSE.csv)
✅ Environment Loaded.

🔍 Scanning dataset for an active Elephant Flow...
   🎯 Found Elephant Flow: Host 4 -> Host 15 (113.23 Mbps)
   📍 Targeting Edge Switch: 11

   [Time]  |  Switch 11 Port 1 (Path A)  |  Switch 11 Port 2 (Path B)  |  Controller Action
   -------------------------------------------------------------------------------------
   [01s]   |           73.5%          |          158.6%          |  
   [02s]   |          100.3%          |           19.8%          |  
   [03s]   |            0.1%          |          102.3%          |  
   [04s]   |           18.8%          |            5.3%          |  
   [05s]   |           10.1%          |           16.9%          |  
   [06s]   |           76.8%          |           34.9%          |  🚨 REROUTE COMMAND SENT!
   [07s]   |           79.6%          |           75.9%          |  
   [08s]   |            3.7%          |       